In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd
import numpy as np

In [45]:
Historical_DOU_Fall_Counts = pd.read_csv("2021_Q1-Q2 TEC Ambulatory Fall Summary.csv")
Q1 = Historical_DOU_Fall_Counts[Historical_DOU_Fall_Counts.Month.isin([1,2,3])]
Q2 = Historical_DOU_Fall_Counts[Historical_DOU_Fall_Counts.Month.isin([4,5,6])]
Q3 = Historical_DOU_Fall_Counts[Historical_DOU_Fall_Counts.Month.isin([7,8,9])]
Q4 = Historical_DOU_Fall_Counts[Historical_DOU_Fall_Counts.Month.isin([10,11,12])]

In [46]:
Historical_DOU_Fall_Details = pd.read_csv("2021_Q1-Q2 TEC Ambulatory Fall Details.csv")
Q1_Details = Historical_DOU_Fall_Details[Historical_DOU_Fall_Details.Month.isin([1,2,3])]
Q2_Details = Historical_DOU_Fall_Details[Historical_DOU_Fall_Details.Month.isin([4,5,6])]
Q3_Details = Historical_DOU_Fall_Details[Historical_DOU_Fall_Details.Month.isin([7,8,9])]
Q4_Details = Historical_DOU_Fall_Details[Historical_DOU_Fall_Details.Month.isin([10,11,12])]

In [56]:
# This is the overall turnover summary data
Fall_data = Q2
# These are the individual turnover details
Individual_Falls = Q2_Details

In [57]:
# Fill NaN with 0 and change data type from float to integer
Fall_data.loc[:, ('NumberofPatientFalls')] = Fall_data.loc[:, ('NumberofPatientFalls')].fillna(0.0).astype(int)

In [58]:
Fall_data.head()

,Hospital,UnitName,Month,NumberofPatientFalls,NumberofSuspectedIntentionalFallEvents
3,TEC,Surgical Services: Colon and Rectal Surgery at...,4,0,0
4,TEC,Surgical Services: Colon and Rectal Surgery at...,5,0,0
5,TEC,Surgical Services: Colon and Rectal Surgery at...,6,0,0
9,TEC,Dermatology: Dermatology Clifton,4,0,0
10,TEC,Dermatology: Dermatology Clifton,5,0,0


In [59]:
# Change the patient age from float to integer but first make the null age a 0
Individual_Falls.Years = Individual_Falls.Years.fillna(0.0).astype(int)

In [60]:
Individual_Falls.head()

,Hospital,UnitName,Month,PatientGender,Years,Months,Days,TypeofFall,PhysiologicalFall,InjuryLevel,FallAssistedByEmployee,FallAssistedEmployeeType,FallObserved
0,TEC,Surgical Services: Surgery and Surgical Oncolo...,6,Male,63,NaN,NaN,Other,NaN,None,No,NaN,NaN
1,TEC,"Winship Cancer Institute: Hematology Oncology,...",6,Male,83,NaN,NaN,Other,NaN,Minor,No,NaN,NaN
2,TEC,"Heart and Vascular: Heart and Vascular Center,...",6,Female,64,NaN,NaN,Other,NaN,Major,No,NaN,NaN
3,TEC,Infusion Center - Midtown - EUH Midtown,6,Female,77,NaN,NaN,Other,NaN,Minor,No,NaN,NaN
4,TEC,Musculoskeletal: Spine Center EP,6,Female,19,NaN,NaN,Other,NaN,None,No,NaN,Yes


In [10]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Ambulatory_Falls_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    # This section is to update the total Fall Count Values

    # The first step takes only the data for this selected hospital
    Hosp_data = Fall_data[Fall_data.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct NDNQI code, year, and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # This loops through each separate UnitName from the Template XML file and replaces it WITH the correct NDNQI
    # name for the units. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # With this correct EUOSH UnitName grab each of the 3 months for that unit from the EUOSH data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with each listed month from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]

            # Now take each xml month's "Unit Fall Ambulatory Summary" section
            amb_fall_nodes = month.getElementsByTagName("UnitFallAmbulatorySummary")
            for amb_fall_node in amb_fall_nodes:

                # and replace each xml metric value with data that has the same name from our data source
                for column in new_data.columns[3:]:

                    new_value = new_data[column].iloc[idx2]
                    amb_fall_node.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value



    # This is a temporary xml file that I will now use to add the individual falls data.
    tree.writexml(open('Temp_{}.xml'.format(Selected_Hosp), 'w'))

In [11]:
# The variables needed for this function are the: Hospital abbreviation, Hospital Temp XML file,
# Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Amb_Indiv_Falls_XML(Selected_Hosp,Temp_XML,Calendar_Year,Calendar_Quarter):
    # This Hospital's last Quarter's individual falls
    Hosp_individual = Individual_Falls[Individual_Falls.Hospital == Selected_Hosp]
    
    # This section is to insert the individual falls. 

    individual_falls = ['Years', 'Months', 'Days', 'PatientGender', 'TypeofFall', 'PhysiologicalFall',
                        'FallAssistedByEmployee', 'FallAssistedEmployeeType', 'InjuryLevel', 'FallObserved']

    # This is the temporary xml file that has already had the "FallsCount" section filled
    tree = xml.dom.minidom.parse(Temp_XML)
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # This loops through each listed unit in our xml file
    for idx, unit in enumerate(unit_nodes):
        each_unit = unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue

        # Then we loop through each listed month
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            each_month = month.getElementsByTagName('Month')[0].childNodes[0].nodeValue

            # Now take each xml month's "Unit Fall Ambulatory Summary" section
            amb_fall_nodes = month.getElementsByTagName("UnitFallAmbulatorySummary")
            for amb_fall_node in amb_fall_nodes:

                # I first check to find if any falls were listed
                fall_count = month.getElementsByTagName('NumberofPatientFalls')[0].childNodes[0].nodeValue
                intentional_fall_count = month.getElementsByTagName('NumberofSuspectedIntentionalFallEvents')[0].childNodes[0].nodeValue

                all_falls = [int(fall_count), int(intentional_fall_count)]

                total_falls = sum(all_falls)

                # If a fall was listed for that month and that unit
                if total_falls > 0:

                    # Then I grab this unit's data from Hosp_individual falls
                    new_data = Hosp_individual[Hosp_individual.UnitName == each_unit]
                    # Lastly, I select only that unit's separations that occured in this specific month
                    each_fall = new_data[new_data.Month == int(each_month)]

                    # Now I can take each listed fall
                    for i in range(len(each_fall)):
                        # and create a "UnitFall" node for each individual fall
                        fall = tree.createElement("UnitFallAmbulatoryPatient")
                        for data in individual_falls:
                            # I create an element for each individual's metrics
                            data_name = tree.createElement(data)

                            if data == 'Years' and each_fall[data].iloc[i] == 0:
                                data_name.setAttribute('xsi:nil', 'true')
                            elif pd.isna(each_fall[data].iloc[i]):
                                data_name.setAttribute('xsi:nil', 'true')
                            else:
                                # and I add each one to that individual's "UnitFall" node
                                data_name.appendChild(tree.createTextNode(str(each_fall[data].iloc[i])))

                            fall.appendChild(data_name)

                        # Finally, I can add these individual falls to the "UnitFalls" section.
                        month.getElementsByTagName("UnitFallAmbulatoryPatients")[0].appendChild(fall)


    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} Ambulatory Falls.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

## EUOSH

In [9]:
NDNQI_Ambulatory_Falls_XML('EUOSH','2020 EUOSH Ambulatory Falls Template.xml','SBI556',2021,2)

In [10]:
NDNQI_Amb_Indiv_Falls_XML('EUOSH','Temp_EUOSH.xml',2021,2)

In [11]:
# This Hospital's Turnover Summary Data
EUOSH_data = Fall_data[Fall_data.Hospital == 'EUOSH']
# This Hospital's last Quarter's individual falls
EUOSH_individual = Individual_Falls[Individual_Falls.Hospital == 'EUOSH']

In [12]:
EUOSH_data

,Hospital,UnitName,Month,NumberofPatientFalls,NumberofSuspectedIntentionalFallEvents
30,EUOSH,OSH PACU,4,0,0
31,EUOSH,OSH PACU,5,0,0
32,EUOSH,OSH PACU,6,0,0
75,EUOSH,OSH POHA/PAT,4,0,0
76,EUOSH,OSH POHA/PAT,5,0,0
77,EUOSH,OSH POHA/PAT,6,0,0


In [13]:
EUOSH_individual

,Hospital,UnitName,Month,Years,Months,Days,PatientGender,TypeofFall,PhysiologicalFall,FallAssistedByEmployee,FallAssistedEmployeeType,InjuryLevel,FallObserved


## EJCH

In [14]:
NDNQI_Ambulatory_Falls_XML('EJCH','2020 EJCH Ambulatory Falls Template.xml','YTD069',2021,2)

In [15]:
NDNQI_Amb_Indiv_Falls_XML('EJCH','Temp_EJCH.xml',2021,2)

In [16]:
# This Hospital's Turnover Summary Data
EJCH_data = Fall_data[Fall_data.Hospital == 'EJCH']
# This Hospital's last Quarter's individual falls
EJCH_individual = Individual_Falls[Individual_Falls.Hospital == 'EJCH']

In [17]:
EJCH_data

,Hospital,UnitName,Month,NumberofPatientFalls,NumberofSuspectedIntentionalFallEvents
18,EJCH,JCPain,4,0,0
19,EJCH,JCPain,5,0,0
20,EJCH,JCPain,6,0,0
21,EJCH,JC SDS,4,0,0
22,EJCH,JC SDS,5,0,0
23,EJCH,JC SDS,6,0,0
66,EJCH,JC ROCU,4,0,0
67,EJCH,JC ROCU,5,0,0
68,EJCH,JC ROCU,6,0,0
84,EJCH,JCEndo,4,0,0


In [18]:
EJCH_individual

,Hospital,UnitName,Month,Years,Months,Days,PatientGender,TypeofFall,PhysiologicalFall,FallAssistedByEmployee,FallAssistedEmployeeType,InjuryLevel,FallObserved
6,EJCH,JC ED,5,56,NaN,NaN,Female,Other,NaN,Yes,NaN,Major,Yes
7,EJCH,JC ED,5,50,NaN,NaN,Female,Other,NaN,No,NaN,Moderate,Yes
10,EJCH,JC CDU,5,84,NaN,NaN,Female,Other,NaN,No,NaN,None,NaN
14,EJCH,JC ED,5,60,NaN,NaN,Male,Other,NaN,No,NaN,None,NaN


## ESJH

In [19]:
NDNQI_Ambulatory_Falls_XML('ESJH','2020 ESJH Ambulatory Falls Template.xml','XAR317',2021,2)

In [20]:
NDNQI_Amb_Indiv_Falls_XML('ESJH','Temp_ESJH.xml',2021,2)

In [21]:
# This Hospital's Turnover Summary Data
ESJH_data = Fall_data[Fall_data.Hospital == 'ESJH']
# This Hospital's last Quarter's individual falls
ESJH_individual = Individual_Falls[Individual_Falls.Hospital == 'ESJH']

In [22]:
ESJH_individual

,Hospital,UnitName,Month,Years,Months,Days,PatientGender,TypeofFall,PhysiologicalFall,FallAssistedByEmployee,FallAssistedEmployeeType,InjuryLevel,FallObserved
0,ESJH,ESJH Emergency Services,6,41,NaN,NaN,Female,Other,NaN,No,NaN,None,Yes
1,ESJH,ESJH Emergency Services,6,27,NaN,NaN,Male,Other,NaN,No,NaN,None,Yes
2,ESJH,ESJH Emergency Services,6,27,NaN,NaN,Male,Other,NaN,No,NaN,Minor,Yes
5,ESJH,1 West (Extended observation unit),6,86,NaN,NaN,Male,Other,NaN,No,NaN,None,NaN
9,ESJH,1 West (Extended observation unit),5,55,NaN,NaN,Male,Other,NaN,No,NaN,None,NaN
12,ESJH,1 West (Extended observation unit),5,90,NaN,NaN,Male,Other,NaN,No,NaN,None,Yes
17,ESJH,"ESJH Pre-Postoperative (PAT, Pre-Op, PSCU, Gro...",4,62,NaN,NaN,Female,Other,NaN,No,NaN,None,Yes
18,ESJH,ESJH Emergency Services,4,72,NaN,NaN,Female,Other,NaN,No,NaN,None,Yes
20,ESJH,ESJH Radiology Special Procedures/Outpatient R...,4,30,NaN,NaN,Female,Other,NaN,No,NaN,Minor,NaN
23,ESJH,ESJH Emergency Services,4,86,NaN,NaN,Male,Other,NaN,No,NaN,None,Yes


## EUHM

In [23]:
NDNQI_Ambulatory_Falls_XML('EUHM','2020 EUHM Ambulatory Falls Template.xml','PVF155',2021,2)

In [24]:
NDNQI_Amb_Indiv_Falls_XML('EUHM','Temp_EUHM.xml',2021,2)

In [25]:
# This Hospital's Turnover Summary Data
EUHM_data = Fall_data[Fall_data.Hospital == 'EUHM']
# This Hospital's last Quarter's individual falls
EUHM_individual = Individual_Falls[Individual_Falls.Hospital == 'EUHM']

In [26]:
EUHM_individual

,Hospital,UnitName,Month,Years,Months,Days,PatientGender,TypeofFall,PhysiologicalFall,FallAssistedByEmployee,FallAssistedEmployeeType,InjuryLevel,FallObserved
3,EUHM,EUHM ED,6,53,NaN,NaN,Female,Other,NaN,No,NaN,None,NaN
15,EUHM,EUHM ED,5,66,NaN,NaN,Female,Other,NaN,No,NaN,None,NaN
19,EUHM,EUHM CDU,4,72,NaN,NaN,Male,Other,NaN,No,NaN,None,NaN
21,EUHM,EUHM ED,4,51,NaN,NaN,Female,Other,NaN,No,NaN,None,NaN
24,EUHM,EUHM ED,4,77,NaN,NaN,Female,Other,NaN,No,NaN,Minor,Yes


## EUH

In [27]:
NDNQI_Ambulatory_Falls_XML('EUH','2020 EUH Ambulatory Falls Template.xml','JXE432',2021,2)

In [28]:
NDNQI_Amb_Indiv_Falls_XML('EUH','Temp_EUH.xml',2021,2)

In [29]:
# This Hospital's Turnover Summary Data
EUH_data = Fall_data[Fall_data.Hospital == 'EUH']
# This Hospital's last Quarter's individual falls
EUH_individual = Individual_Falls[Individual_Falls.Hospital == 'EUH']

In [30]:
EUH_individual

,Hospital,UnitName,Month,Years,Months,Days,PatientGender,TypeofFall,PhysiologicalFall,FallAssistedByEmployee,FallAssistedEmployeeType,InjuryLevel,FallObserved
4,EUH,EUH ED,6,61,NaN,NaN,Female,Other,NaN,No,NaN,None,NaN
8,EUH,EUH ED,5,77,NaN,NaN,Female,Other,NaN,No,NaN,None,NaN
11,EUH,EUH ED,5,90,NaN,NaN,Female,Other,NaN,No,NaN,Minor,NaN
13,EUH,EUH ED,5,53,NaN,NaN,Male,Other,NaN,No,NaN,Minor,Yes
16,EUH,EUH POHA,5,33,NaN,NaN,Female,Other,NaN,Yes,Nursing Staff,None,Yes
22,EUH,EUH ED,4,66,NaN,NaN,Male,Other,NaN,No,NaN,None,NaN


## EDH

In [38]:
NDNQI_Ambulatory_Falls_XML('EDH','2021 EDH Ambulatory Falls Template.xml','NMW346',2021,2)

In [39]:
NDNQI_Amb_Indiv_Falls_XML('EDH','Temp_EDH.xml',2021,2)

In [40]:
# This Hospital's Turnover Summary Data
EDH_data = Fall_data[Fall_data.Hospital == 'EDH']
# This Hospital's last Quarter's individual falls
EDH_individual = Individual_Falls[Individual_Falls.Hospital == 'EDH']

In [41]:
EDH_individual

,Hospital,UnitName,Month,Years,Months,Days,TypeofFall,PhysiologicalFall,InjuryLevel,FallAssistedByEmployee,FallObserved,PatientGender,FallAssistedEmployeeType
2,EDH,EDH ED,4,0,NaN,NaN,Other,NaN,None,No,NaN,Female,NaN
3,EDH,EDH ED,6,80,NaN,NaN,Other,NaN,Minor,No,NaN,Male,NaN


## EHH

In [42]:
NDNQI_Ambulatory_Falls_XML('EHH','2021 EHH Ambulatory Falls Template.xml','WXX116',2021,2)

In [43]:
NDNQI_Amb_Indiv_Falls_XML('EHH','Temp_EHH.xml',2021,2)

In [44]:
# This Hospital's Turnover Summary Data
EHH_data = Fall_data[Fall_data.Hospital == 'EHH']
# This Hospital's last Quarter's individual falls
EHH_individual = Individual_Falls[Individual_Falls.Hospital == 'EHH']

In [45]:
EHH_individual

,Hospital,UnitName,Month,Years,Months,Days,TypeofFall,PhysiologicalFall,InjuryLevel,FallAssistedByEmployee,FallObserved,PatientGender,FallAssistedEmployeeType
0,EHH,EHH ED,6,46,NaN,NaN,Other,NaN,None,Yes,Yes,Male,Nursing Staff
1,EHH,EHH ED,6,82,NaN,NaN,Other,NaN,None,No,NaN,Female,NaN


## TEC

In [61]:
NDNQI_Ambulatory_Falls_XML('TEC','2021 TEC Ambulatory Falls Template.xml','NLH637',2021,2)

In [62]:
NDNQI_Amb_Indiv_Falls_XML('TEC','Temp_TEC.xml',2021,2)

In [63]:
# This Hospital's Turnover Summary Data
TEC_data = Fall_data[Fall_data.Hospital == 'TEC']
# This Hospital's last Quarter's individual falls
TEC_individual = Individual_Falls[Individual_Falls.Hospital == 'TEC']

In [64]:
TEC_individual

,Hospital,UnitName,Month,PatientGender,Years,Months,Days,TypeofFall,PhysiologicalFall,InjuryLevel,FallAssistedByEmployee,FallAssistedEmployeeType,FallObserved
0,TEC,Surgical Services: Surgery and Surgical Oncolo...,6,Male,63,NaN,NaN,Other,NaN,None,No,NaN,NaN
1,TEC,"Winship Cancer Institute: Hematology Oncology,...",6,Male,83,NaN,NaN,Other,NaN,Minor,No,NaN,NaN
2,TEC,"Heart and Vascular: Heart and Vascular Center,...",6,Female,64,NaN,NaN,Other,NaN,Major,No,NaN,NaN
3,TEC,Infusion Center - Midtown - EUH Midtown,6,Female,77,NaN,NaN,Other,NaN,Minor,No,NaN,NaN
4,TEC,Musculoskeletal: Spine Center EP,6,Female,19,NaN,NaN,Other,NaN,None,No,NaN,Yes
5,TEC,Infusion Center - Winship TEC - Building C - INFE,6,Female,57,NaN,NaN,Other,NaN,None,No,NaN,NaN
9,TEC,Surgical Services: ENT and Facial Plastic Surg...,4,Male,89,NaN,NaN,Other,NaN,Minor,No,NaN,NaN
10,TEC,ASC Main - Emory University Hospital,4,Female,61,NaN,NaN,Other,NaN,None,No,NaN,NaN
11,TEC,Musculoskeletal: Orthopaedics: General Ortho a...,4,Female,88,NaN,NaN,Other,NaN,Minor,No,NaN,Yes
12,TEC,Surgical Services: General Surgery St Josephs,5,Female,41,NaN,NaN,Other,NaN,None,No,NaN,Yes
